In [1]:
# Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import random

In [20]:
human = pd.read_table('../datasets/human.txt')

In [21]:
human

,sequence,class
0,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...,4
1,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,4
2,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
3,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
4,ATGCAACAGCATTTTGAATTTGAATACCAGACCAAAGTGGATGGTG...,3
...,...,...
4375,ATGGAAGATTTGGAGGAAACATTATTTGAAGAATTTGAAAACTATT...,0
4376,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4377,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4378,ATGGGGCACCTGGTTTGCTGTCTGTGTGGCAAGTGGGCCAGTTACC...,6


In [22]:
#lets descripe one dna sequence of human
one_dna = np.array(human.head(1))
one_dna = str(one_dna).split(' ')
seq = one_dna[0].replace("[['",'')
seq = seq.replace("'",'')
print(seq)

ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCATACTCCTTACACTATTCCTCATCACCCAACTAAAAATATTAAACACAAACTACCACCTACCTCCCTCACCAAAGCCCATAAAAATAAAAAATTATAACAAACCCTGAGAACCAAAATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG



In [23]:
#create list of lovercase char and check if any char not in (a, c, g, t) put it as (z) char
def string_to_array(seq_string):
    seq_string = seq_string.lower()
    seq_string = re.sub('[^acgt]', 'z', seq_string)
    seq_string = np.array(list(seq_string))
    return seq_string
string_to_array(seq)

array(['a', 't', 'g', 'c', 'c', 'c', 'c', 'a', 'a', 'c', 't', 'a', 'a',
       'a', 't', 'a', 'c', 't', 'a', 'c', 'c', 'g', 't', 'a', 't', 'g',
       'g', 'c', 'c', 'c', 'a', 'c', 'c', 'a', 't', 'a', 'a', 't', 't',
       'a', 'c', 'c', 'c', 'c', 'c', 'a', 't', 'a', 'c', 't', 'c', 'c',
       't', 't', 'a', 'c', 'a', 'c', 't', 'a', 't', 't', 'c', 'c', 't',
       'c', 'a', 't', 'c', 'a', 'c', 'c', 'c', 'a', 'a', 'c', 't', 'a',
       'a', 'a', 'a', 'a', 't', 'a', 't', 't', 'a', 'a', 'a', 'c', 'a',
       'c', 'a', 'a', 'a', 'c', 't', 'a', 'c', 'c', 'a', 'c', 'c', 't',
       'a', 'c', 'c', 't', 'c', 'c', 'c', 't', 'c', 'a', 'c', 'c', 'a',
       'a', 'a', 'g', 'c', 'c', 'c', 'a', 't', 'a', 'a', 'a', 'a', 'a',
       't', 'a', 'a', 'a', 'a', 'a', 'a', 't', 't', 'a', 't', 'a', 'a',
       'c', 'a', 'a', 'a', 'c', 'c', 'c', 't', 'g', 'a', 'g', 'a', 'a',
       'c', 'c', 'a', 'a', 'a', 'a', 't', 'g', 'a', 'a', 'c', 'g', 'a',
       'a', 'a', 'a', 't', 'c', 't', 'g', 't', 't', 'c', 'g', 'c

In [24]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def one_hot_encoder(seq_string):
    label_encoder = LabelEncoder()
    label_encoder.fit(np.array(['a','c','g','t','z']))
    int_encoded = label_encoder.transform(seq_string)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)
    int_encoded = int_encoded.reshape(len(int_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(int_encoded)
    return onehot_encoded
#So let’s try it out with a simple short sequence:
seq_test = 'attcgxffgtg'
one_hot_encoder(string_to_array(seq_test))

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]])

In [25]:
def Kmers_funct(seq, size=6):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]


#So let’s try it out with a simple sequence:
mySeq = 'GTGCCCAGGTTCAGTGAGTGACACAGGCAG'
#size of each sub-seq = 6 
words = Kmers_funct(mySeq, size=6)
print(words)

['gtgccc', 'tgccca', 'gcccag', 'cccagg', 'ccaggt', 'caggtt', 'aggttc', 'ggttca', 'gttcag', 'ttcagt', 'tcagtg', 'cagtga', 'agtgag', 'gtgagt', 'tgagtg', 'gagtga', 'agtgac', 'gtgaca', 'tgacac', 'gacaca', 'acacag', 'cacagg', 'acaggc', 'caggca', 'aggcag']


In [26]:
#change words to sentences by concatenates all words
joined_sentence = ' '.join(words)
joined_sentence

'gtgccc tgccca gcccag cccagg ccaggt caggtt aggttc ggttca gttcag ttcagt tcagtg cagtga agtgag gtgagt tgagtg gagtga agtgac gtgaca tgacac gacaca acacag cacagg acaggc caggca aggcag'

In [28]:
#apply kmers function to all datasets & join all words to list 
#then add words column & drop sequence column
human['words'] = human.apply(lambda x: Kmers_funct(x['sequence']), axis=1)
human = human.drop('sequence', axis=1)

human.head()

,class,words
0,4,"[atgccc, tgcccc, gcccca, ccccaa, cccaac, ccaac..."
1,4,"[atgaac, tgaacg, gaacga, aacgaa, acgaaa, cgaaa..."
2,3,"[atgtgt, tgtgtg, gtgtgg, tgtggc, gtggca, tggca..."
3,3,"[atgtgt, tgtgtg, gtgtgg, tgtggc, gtggca, tggca..."
4,3,"[atgcaa, tgcaac, gcaaca, caacag, aacagc, acagc..."


In [31]:
#apply to human
human_texts = list(human['words'])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
#separate labels
y_human = human.iloc[:, 0].values # y_human for human_dna


print(y_human)

[4 4 3 ... 6 6 6]


In [47]:
y_human[0:5],human_texts[0]

(array([4, 4, 3, 3, 3]),
 'atgccc tgcccc gcccca ccccaa cccaac ccaact caacta aactaa actaaa ctaaat taaata aaatac aatact atacta tactac actacc ctaccg taccgt accgta ccgtat cgtatg gtatgg tatggc atggcc tggccc ggccca gcccac cccacc ccacca caccat accata ccataa cataat ataatt taatta aattac attacc ttaccc tacccc accccc ccccca ccccat cccata ccatac catact atactc tactcc actcct ctcctt tcctta ccttac cttaca ttacac tacact acacta cactat actatt ctattc tattcc attcct ttcctc tcctca cctcat ctcatc tcatca catcac atcacc tcaccc caccca acccaa cccaac ccaact caacta aactaa actaaa ctaaaa taaaaa aaaaat aaaata aaatat aatatt atatta tattaa attaaa ttaaac taaaca aaacac aacaca acacaa cacaaa acaaac caaact aaacta aactac actacc ctacca taccac accacc ccacct caccta acctac cctacc ctacct tacctc acctcc cctccc ctccct tccctc ccctca cctcac ctcacc tcacca caccaa accaaa ccaaag caaagc aaagcc aagccc agccca gcccat cccata ccataa cataaa ataaaa taaaaa aaaaat aaaata aaataa aataaa ataaaa taaaaa aaaaaa aaaaat aaaatt aaatta aattat attata ttataa tataac 

In [59]:
cv = CountVectorizer() #The n-gram size of 4 is previously determined by testing
X_human = cv.fit_transform(human_texts)

print(X_human.shape)


(4380, 4469)


In [60]:
print (X_human[0])

  (0, 917)	1
  (0, 4042)	2
  (0, 2388)	1
  (0, 1360)	1
  (0, 1345)	2
  (0, 1287)	2
  (0, 1052)	2
  (0, 112)	2
  (0, 448)	2
  (0, 1795)	1
  (0, 3457)	1
  (0, 49)	1
  (0, 199)	1
  (0, 796)	1
  (0, 3558)	1
  (0, 453)	2
  (0, 1814)	1
  (0, 3536)	1
  (0, 364)	1
  (0, 1459)	1
  (0, 1742)	1
  (0, 2874)	1
  (0, 3678)	1
  (0, 933)	1
  (0, 4106)	1
  :	:
  (0, 4203)	1
  (0, 3033)	1
  (0, 4316)	1
  (0, 3860)	1
  (0, 1661)	1
  (0, 2548)	1
  (0, 2003)	1
  (0, 4292)	2
  (0, 3762)	1
  (0, 1268)	1
  (0, 979)	1
  (0, 3763)	1
  (0, 1273)	1
  (0, 997)	1
  (0, 4362)	1
  (0, 2389)	1
  (0, 1361)	1
  (0, 1348)	1
  (0, 1296)	1
  (0, 1091)	1
  (0, 269)	1
  (0, 1077)	1
  (0, 215)	1
  (0, 860)	1
  (0, 3815)	1


In [61]:
# Splitting the human dataset into the training set and test set with (80% train & 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_human, 
                                                    y_human, 
                                                    test_size = 0.20, 
                                                    random_state=42)

In [62]:
#The fit method of KNN class is called to train the algorithm on the training data, 
#which is passed as a parameter to the fit method
from sklearn.neighbors import KNeighborsClassifier
KNNclassifier = KNeighborsClassifier(n_neighbors=5)
KNNclassifier.fit(X_train, y_train)

KNeighborsClassifier()

In [63]:
# enter all test datatest in model 
y_pred = KNNclassifier.predict(X_test) 

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 80   2   1   6   1   0  12]
 [  3  80   0   1   1   1  20]
 [  2   4  60   1   0   0  11]
 [  5   3   0  98   3   0  16]
 [  7   5   1   2 104   2  28]
 [  6   3   0   0   2  32   8]
 [  8   6   2   4   5   0 240]]
              precision    recall  f1-score   support

           0       0.72      0.78      0.75       102
           1       0.78      0.75      0.77       106
           2       0.94      0.77      0.85        78
           3       0.88      0.78      0.83       125
           4       0.90      0.70      0.78       149
           5       0.91      0.63      0.74        51
           6       0.72      0.91      0.80       265

    accuracy                           0.79       876
   macro avg       0.83      0.76      0.79       876
weighted avg       0.81      0.79      0.79       876

